# Average Creature Stats by Mana Value

In this notebook i want to make my first attempt at understanding how creature stats can influence performance of a card in a limited environment. To do this, i chose to begin with the set *Adventures in the Forgotten Realms* (AFR for short) as it is a very simple set (no double sided cards).

## Adquiring the Data

The data i'll be using comes from [Scryfall's API](https://scryfall.com/docs/api) theres not much thought into this decision it's just the mtg data source that i like and know how to use. To access the data i wrote a python class that will handle the pagination of the query result immediately.

```
import requests
from time import sleep

class ScryfallCardQuery_v2:
    def __init__(self, query_string):
        self.response = requests.get('https://api.scryfall.com/cards/search?q=' + query_string).json()
        self.data = self.response['data']
        sleep(0.1)
        if self.response['has_more']:
            self.data += self.get_next_pages(self.response['next_page']) 
    
    def get_next_pages(self, next_page):
        next_data = requests.get(next_page).json()

        if next_data['has_more']:
            sleep(0.1)
            return next_data['data'] + self.get_next_pages(next_data['next_page'])
        else:
            return next_data['data']
```

To make a query with this class you just have to import it and send the scryfall query string as an argument. Given i want to see the creatures in AFR then my query should be: 

 `type:creature set:afr -is:rebalanced` 

the last bit of the query is required to prevent the alchemy rebalanced version of cards from popping up in the results.

 In addition to this, i want this resulting data to be stored as a pandas DataFrame which is designed for data manipulation

In [5]:
import pandas as pd
from ScryfallCardQuery import ScryfallCardQuery_v2 as scq

afr_query = 'type:creature set:afr -is:rebalanced' 
afr_creature_full_data = pd.DataFrame(scq(afr_query).data)

afr_creature_full_data.head()

,object,id,oracle_id,multiverse_ids,mtgo_id,arena_id,tcgplayer_id,cardmarket_id,name,lang,...,preview,prices,related_uris,purchase_uris,all_parts,frame_effects,security_stamp,flavor_text,penny_rank,produced_mana
0,card,d9c6401a-e152-4acf-a2ca-75c484f296d5,753ead8c-f3f8-42fe-931d-79f5ee3ece54,[527331],91590,77149,243580,571887,Aberrant Mind Sorcerer,en,...,"{'source': 'Neo Tokyo Project', 'source_uri': ...","{'usd': '0.04', 'usd_foil': '0.09', 'usd_etche...",{'gatherer': 'https://gatherer.wizards.com/Pag...,{'tcgplayer': 'https://www.tcgplayer.com/produ...,NaN,NaN,NaN,NaN,NaN,NaN
1,card,dd52d0bd-3abd-401c-9f56-ee911613da3b,31b4e85d-89fc-4712-8f05-7ec097b1d3b6,[527374],91676,77192,243442,571848,Acererak the Archlich,en,...,"{'source': 'iYingdi', 'source_uri': 'https://w...","{'usd': '6.48', 'usd_foil': '6.19', 'usd_etche...",{'gatherer': 'https://gatherer.wizards.com/Pag...,{'tcgplayer': 'https://www.tcgplayer.com/produ...,"[{'object': 'related_card', 'id': 'eb363654-20...",[legendary],oval,NaN,NaN,NaN
2,card,67a76010-d932-4727-8b5e-b8e2d14e1362,a361fdd1-7f97-4aed-8e24-ab2a03387f73,[527503],91934,77321,243083,571085,Adult Gold Dragon,en,...,"{'source': 'Wizards of the Coast', 'source_uri...","{'usd': '0.32', 'usd_foil': '0.34', 'usd_etche...",{'gatherer': 'https://gatherer.wizards.com/Pag...,{'tcgplayer': 'https://www.tcgplayer.com/produ...,NaN,NaN,oval,Gold dragons are the greatest of the metallic ...,3334.0,NaN
3,card,fba82564-a9a9-4ded-a5fa-0e5fab2a3faa,6ec2d957-c6d9-469f-8190-8b621c490238,[527332],91592,77150,243149,571242,Air-Cult Elemental,en,...,NaN,"{'usd': '0.01', 'usd_foil': '0.05', 'usd_etche...",{'gatherer': 'https://gatherer.wizards.com/Pag...,{'tcgplayer': 'https://www.tcgplayer.com/produ...,NaN,NaN,NaN,"Elemental Evil distorts the natural elements, ...",NaN,NaN
4,card,fc45c9d4-ecc7-4a9d-9efe-f4b7d697dd97,9a17452d-3adb-42ec-99f8-0083740cefa0,[527289],91506,77107,243434,571841,Arborea Pegasus,en,...,"{'source': 'Wizards of the Coast', 'source_uri...","{'usd': '0.01', 'usd_foil': '0.04', 'usd_etche...",{'gatherer': 'https://gatherer.wizards.com/Pag...,{'tcgplayer': 'https://www.tcgplayer.com/produ...,NaN,NaN,NaN,The noble steeds of the elven gods can sometim...,NaN,NaN


Well thats a lot of data! in this query for every card there are 69 tracked features. 

In [7]:
afr_creature_full_data.columns

Index(['object', 'id', 'oracle_id', 'multiverse_ids', 'mtgo_id', 'arena_id',
       'tcgplayer_id', 'cardmarket_id', 'name', 'lang', 'released_at', 'uri',
       'scryfall_uri', 'layout', 'highres_image', 'image_status', 'image_uris',
       'mana_cost', 'cmc', 'type_line', 'oracle_text', 'power', 'toughness',
       'colors', 'color_identity', 'keywords', 'legalities', 'games',
       'reserved', 'foil', 'nonfoil', 'finishes', 'oversized', 'promo',
       'reprint', 'variation', 'set_id', 'set', 'set_name', 'set_type',
       'set_uri', 'set_search_uri', 'scryfall_set_uri', 'rulings_uri',
       'prints_search_uri', 'collector_number', 'digital', 'rarity',
       'card_back_id', 'artist', 'artist_ids', 'illustration_id',
       'border_color', 'frame', 'full_art', 'textless', 'booster',
       'story_spotlight', 'edhrec_rank', 'preview', 'prices', 'related_uris',
       'purchase_uris', 'all_parts', 'frame_effects', 'security_stamp',
       'flavor_text', 'penny_rank', 'produced_mana'

But we are not really interested in all features. For now, we'll be looking at the most basic of features surrounding combat and visualization: card name, mana value (cmc), colors, power and toughness. The image uris will be added so that i can display them in this notebook.

In [51]:
features = ['name', 'cmc', 'power', 'toughness', 'colors', 'image_uris']
afr_creature_data = afr_creature_full_data[features]
afr_creature_data.head()

,name,cmc,power,toughness,colors,image_uris
0,Aberrant Mind Sorcerer,5.0,3,4,[U],{'small': 'https://c1.scryfall.com/file/scryfa...
1,Acererak the Archlich,3.0,5,5,[B],{'small': 'https://c1.scryfall.com/file/scryfa...
2,Adult Gold Dragon,5.0,4,3,"[R, W]",{'small': 'https://c1.scryfall.com/file/scryfa...
3,Air-Cult Elemental,6.0,2,5,[U],{'small': 'https://c1.scryfall.com/file/scryfa...
4,Arborea Pegasus,4.0,2,3,[W],{'small': 'https://c1.scryfall.com/file/scryfa...


That looks good! but we can still do better as with the way the statline of creatures (power and toughness) is sorted out right we can only interact with them individually. The problem being that when we are assesing the quality of a creature and comparing them to one another both stats are a packaged deal. In order to aquire this relationship and additional column (or pandas Series) should be added where both stats are bundled as a string making it a categorical feature.

In [52]:
afr_creature_statline = pd.Series([str(card['power'])+'/'+str(card['toughness'])
                                                for i, card in afr_creature_data.iterrows()]).rename('statline')

afr_creature_data = pd.concat([afr_creature_data, afr_creature_statline], axis='columns')
afr_creature_data.head()


,name,cmc,power,toughness,colors,image_uris,statline
0,Aberrant Mind Sorcerer,5.0,3,4,[U],{'small': 'https://c1.scryfall.com/file/scryfa...,3/4
1,Acererak the Archlich,3.0,5,5,[B],{'small': 'https://c1.scryfall.com/file/scryfa...,5/5
2,Adult Gold Dragon,5.0,4,3,"[R, W]",{'small': 'https://c1.scryfall.com/file/scryfa...,4/3
3,Air-Cult Elemental,6.0,2,5,[U],{'small': 'https://c1.scryfall.com/file/scryfa...,2/5
4,Arborea Pegasus,4.0,2,3,[W],{'small': 'https://c1.scryfall.com/file/scryfa...,2/3
